In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import colorcet as cc
import ipywidgets as widgets

%run _helpers.py

sns.set_theme()
pd.set_option('display.max_rows', 80)
idx = pd.IndexSlice

gdf_clean = gpd.read_parquet('../data/processed/clean.parquet')
print(f"loaded {gdf_clean.shape[0]} datapoints")

In [2]:
gdf_population = gpd.read_parquet('../data/processed/subset_territorial_season.parquet')
gdf_night = gpd.read_parquet('../data/processed/subset_cohorts_juvenile_nights_timeshifted.parquet')
gdf_natal_candidates = gpd.read_parquet('../data/processed/areas_natal_territory_candidates.parquet')

In [ ]:
gdf_natal_candidates.loc[idx[:,1],:]

In [ ]:
w_ind = widgets.Dropdown(options=gdf_natal_candidates.index.levels[0].to_list())
w_ind

In [ ]:
# show clusters and juvenile night positions

ind = w_ind.value

gdf_areas = gdf_natal_candidates.loc[idx[ind,:],:]
gdf_ind = gdf_night.loc[idx[ind,:],:]
cohort = gdf_areas.year.iloc[0]

display(gdf_areas)
m = gdf_areas.reset_index().explore(color='orange')
equipped = gdf_ind.index.get_level_values(1).min()
age = (gdf_ind.index.get_level_values(1) - equipped).days
gdf_ind.reset_index().explore(column=age, cmap='viridis', m=m)

In [ ]:
# show full juvenile data

gdf = gdf_clean.loc[idx[ind,:],:]
gdf = gdf.loc[gdf.age == 0.]

fig, axes = plt.subplot_mosaic('A', figsize=(8,2))
sns.scatterplot(gdf, x='ts', y='ind', marker='.', color='k', linewidth=0., ax=axes['A'])
axes['A'].set_xlabel(None)
axes['A'].set_ylabel(None)
fig.tight_layout()
plt.show()

m = gdf_areas.reset_index().explore(color='orange')
_expl(gdf, m=m)

In [ ]:
# show co-occurrence on clusters
  
gdf_areas = gdf_natal_candidates.loc[idx[ind,:],:]
gdf = gdf_population.loc[gdf_population.index.get_level_values(1).year == cohort]
gdf = gdf.sjoin(gdf_areas).rename_axis(['ind','ts'])

display(gdf.groupby(level=0).lat.count())

fig, axes = plt.subplot_mosaic('A', figsize=(10,6))
sns.scatterplot(gdf, x='ts', y='ind', marker='.', hue=gdf['rank'], palette=cc.cm.glasbey, linewidth=0., ax=axes['A'])
axes['A'].set_xlabel(None)
axes['A'].set_ylabel(None)
fig.tight_layout()
plt.show()

m = gdf_areas.reset_index().explore(color='orange')
gdf.loc[gdf.index.get_level_values(0) != gdf_areas.index.get_level_values(0)[0]].reset_index().explore(column='ind', cmap=cc.cm.glasbey, m=m)